Yolo v8 기본 실습

In [ ]:
# YOLOv8 설치 (Ultralytics)
!pip install -q ultralytics

import torch
import cv2
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import requests
from io import BytesIO
from ultralytics import YOLO

print(f'PyTorch: {torch.__version__}')
print(f'Device: {"cuda" if torch.cuda.is_available() else "cpu"}')

In [ ]:
# 사전학습된 COCO 모델 다운로드 (자동)
# n: nano (가장 빠름), s: small, m: medium, l: large, x: extra large
model = YOLO('yolov8n.pt')  # YOLOv8 Nano 모델

In [ ]:
model

In [ ]:
# sum(p.numel() for p in model.model.parameters())

print(f'모델 파라미터 수: {sum(p.numel() for p in model.model.parameters()):,}')

In [ ]:
# COCO 클래스 이름(80개)
model.names

In [ ]:
class_names = model.names
print(f"총 클래스 수: {len(class_names)}")

In [ ]:
# list(class_names.values())[:10]
print(list(class_names.values())[:10])

In [ ]:
url_test = "https://ultralytics.com/images/bus.jpg"

res = requests.get(url_test)
print(res)  # <Response [200]> 정상 수신
print(res.content)
print(BytesIO(res.content))
print(Image.open(BytesIO(res.content)))
img = Image.open(BytesIO(res.content)).convert('RGB')

In [ ]:
# 테스트 이미지 준비 ================================
def download_image(url):
    """URL에서 이미지 다운로드"""
    try:
        response = requests.get(url, timeout=10)
        img = Image.open(BytesIO(response.content)).convert('RGB')
        return img
    except Exception as e:
        print(f"이미지 다운로드 실패: {e}")
        return None

# 테스트 이미지 URL 목록
test_images = {
    'street': 'https://ultralytics.com/images/bus.jpg',
    'people': 'https://ultralytics.com/images/zidane.jpg',
    'animals': 'https://images.unsplash.com/photo-1583337130417-3346a1be7dee?w=640',
}

print("\n=== 테스트 이미지 다운로드 ===")
images = {}
for name, url in test_images.items():
    print(f"{name} 이미지 다운로드 중...")
    img = download_image(url)
    if img is not None:
        images[name] = img
        print(f"  {name}: {img.size}")


# 샘플 이미지 시각화
if images:
   fig, axes = plt.subplots(1, len(images), figsize=(15,5))
   if len(images) == 1:
      axes = [axes]
   # Tip : 진짜 이미지가 1개면 곤란해 (반복문이 안돌아가요)

   for ax, (name, img) in zip(axes, images.items()):
      ax.imshow(img)
      ax.set_title(f'{name.capitalize()} Image', fontsize=12, fontweight='bold')
      # capitalize : 첫 글자만 대문자로 벼환
      ax.axis('off')

   plt.tight_layout()
   plt.show()


In [ ]:
# 기본 객체 검출

# 첫번째 이미지로 검출
# list(images.keys())[0]
img_name = list(images.keys())[0]
# images[img_name]
test_img = images[img_name]

In [ ]:
print(img_name)
print()
print(test_img)
print()
images[img_name]

In [ ]:
# Yolo8 추론
model(test_img)

In [ ]:
results = model(test_img)

# 결과 추출
results[0] # 첫번째 결과

In [ ]:
results

In [ ]:
result = results[0] # 첫번째 결과
# result.boxes # bbox (bounding box)
boxes = result.boxes # bbox (bounding box)

print(f'검출된 객체수: {len(boxes)}')

In [ ]:
# 검출 결과 상세 정보
if len(boxes) > 0 :
   print("\n검출 상세:")
   for i, box in enumerate(boxes):
      # 박스 정보
      x1, y1, x2, y2 = box.xyxy[0].cpu().numpy()  # tensor >> np.array
      conf = box.conf[0].cpu().item()
      cls = int(box.cls[0].cpu().item())
      class_name = class_names[cls]

      print(f' {i+1}, {class_name}: {conf:.3f} '
            f'[{x1:.0f},{y1:.0f}, {x2:.0f},{y2:.0f}]')

In [ ]:
# 결과 시각화

result_img = result.plot() # BGR 형식

# RGB 로 변환
plt.figure(figsize=(12,8))
plt.imshow(cv2.cvtColor(result_img, cv2.COLOR_BGR2RGB))
plt.axis("off")
plt.title(f'Yolo_detection_Results ({img_name})',
          fontsize=14, fontweight='bold')

plt.tight_layout()
plt.show()

In [ ]:
# 신뢰도 임계값 낮추면 재현율(recall)이 높아진다
# 임계값이 높으면 깐깐함(엄격함)

In [ ]:
# Confidence Threshold 조정

thresholds = [0.25, 0.5, 0.75]
fig, axes = plt.subplots(1, 3, figsize=(18, 6))

for ax, conf_thresh in zip(axes, thresholds):
    # threshold 적용하여 추론
    results_thresh = model(test_img, conf=conf_thresh)
    result_thresh = results_thresh[0]

    # 시각화
    img_plot = result_thresh.plot()
    ax.imshow(cv2.cvtColor(img_plot, cv2.COLOR_BGR2RGB))
    ax.set_title(f'Confidence ≥ {conf_thresh}\n'
                 f'({len(result_thresh.boxes)} objects)',
                 fontsize=12, fontweight='bold')
    ax.axis('off')

plt.tight_layout()
plt.show()


In [ ]:
# 여러 이미지 일괄 검출

fig, axes = plt.subplots(len(images), 2, figsize=(14, 6*len(images)))
if len(images) == 1:
    axes = axes.reshape(1, -1)

for idx, (name, img) in enumerate(images.items()):
    # 원본 이미지
    axes[idx, 0].imshow(img)
    axes[idx, 0].set_title(f'{name.capitalize()} - Original',
                           fontsize=12, fontweight='bold')
    axes[idx, 0].axis('off')

    # 검출 결과
    results = model(img, conf=0.5)
    result_img = results[0].plot()
    axes[idx, 1].imshow(cv2.cvtColor(result_img, cv2.COLOR_BGR2RGB))
    axes[idx, 1].set_title(f'{name.capitalize()} - Detected ({len(results[0].boxes)} objects)',
                           fontsize=12, fontweight='bold')
    axes[idx, 1].axis('off')

plt.tight_layout()
plt.show()

In [ ]:
# 클래스 별 통계분석

# 모든 이미지에서 검출 수행
all_detections = {}

for name, img in images.items():
    results = model(img, conf=0.5)
    boxes = results[0].boxes

    detections = []
    for box in boxes:
        cls = int(box.cls[0].cpu().item())
        conf = box.conf[0].cpu().item()
        detections.append({
            'class': class_names[cls],
            'confidence': conf}
            )
    all_detections[name] = detections

In [ ]:
# 클래스별 카운트
from collections import Counter

for name, detections in all_detections.items():
    if detections:
      class_counts = Counter(d['class'] for d in detections)
      print(class_counts)

      for cls, count in class_counts.most_common():
         avg_conf = np.mean([d['confidence'] for d in detections if d['class'] == cls])
         print(f'{cls}:{count}개(평균 confidence: {avg_conf:.3f})')
else:
    print('검출된 객체 없음')

In [ ]:
# 커스텀 시각화

def draw_custom_boxes(image, results, conf_threshold=0.5):
    """커스텀 박스 그리기"""
    img_np = np.array(image).copy()
    boxes = results[0].boxes

    # 색상 팔레트 (클래스별)
    np.random.seed(42)
    colors = {cls: tuple(np.random.randint(0, 255, 3).tolist())
              for cls in range(len(class_names))}

    for box in boxes:
        conf = box.conf[0].cpu().item()
        if conf < conf_threshold:
            continue

        # 박스 정보
        x1, y1, x2, y2 = box.xyxy[0].cpu().numpy().astype(int)
        cls = int(box.cls[0].cpu().item())
        class_name = class_names[cls]
        color = colors[cls]

        # 박스 그리기
        cv2.rectangle(img_np, (x1, y1), (x2, y2), color, 3)

        # 레이블 배경
        label = f'{class_name} {conf:.2f}'
        (text_w, text_h), _ = cv2.getTextSize(
            label, cv2.FONT_HERSHEY_SIMPLEX, 0.7, 2)
        cv2.rectangle(img_np, (x1, y1 - text_h - 10),
                     (x1 + text_w, y1), color, -1)

        # 레이블 텍스트
        cv2.putText(img_np, label, (x1, y1 - 5),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)

    return img_np

# 커스텀 시각화 적용
img_name = list(images.keys())[0]
test_img = images[img_name]
results = model(test_img)

custom_img = draw_custom_boxes(test_img, results, conf_threshold=0.5)

fig, axes = plt.subplots(1, 2, figsize=(16, 8))
axes[0].imshow(test_img)
axes[0].set_title('Original Image', fontsize=14, fontweight='bold')
axes[0].axis('off')

axes[1].imshow(custom_img)
axes[1].set_title('Custom Visualization', fontsize=14, fontweight='bold')
axes[1].axis('off')

plt.tight_layout()
plt.show()

In [ ]:
# 모델 정보 및 성능 비교

model_info = {
    'YOLOv8n': {'params': '3.2M', 'mAP': 37.3, 'speed': '80+ FPS'},
    'YOLOv8s': {'params': '11.2M', 'mAP': 44.9, 'speed': '50+ FPS'},
    'YOLOv8m': {'params': '25.9M', 'mAP': 50.2, 'speed': '30+ FPS'},
    'YOLOv8l': {'params': '43.7M', 'mAP': 52.9, 'speed': '20+ FPS'},
    'YOLOv8x': {'params': '68.2M', 'mAP': 53.9, 'speed': '15+ FPS'},
}

print("\n| 모델 | 파라미터 | mAP@0.5:0.95 | 속도 (T4 GPU) |")
print("|------|----------|--------------|---------------|")
for model_name, info in model_info.items():
    print(f"| {model_name} | {info['params']} | {info['mAP']}% | {info['speed']} |")

In [ ]:
# 배치 추론 (여러 이미지 동시 처리)

import time

# 이미지 리스트 준비
# list(images.values())
img_list = list(images.values())

# 단일 추론
start_time = time.time()

for img in img_list:
   _ = model(img, verbose=False)
   # 원래 result 에 저장하는 데 여기서는 시간만 측정할려고

single_time = time.time() - start_time
print(single_time)

# 배치 추론
start_time = time.time()
_ = model(img_list, verbose=False)
batch_time = time.time() - start_time
print(batch_time)

In [ ]:
# Colab 파일 업로드 기능
print("\n=== 사용자 이미지 업로드 ===")
print("직접 이미지를 업로드하여 테스트하세요!")

from google.colab import files

print("\n파일 선택 창이 열립니다...")
uploaded = files.upload()

if uploaded:
    # 업로드된 첫 번째 이미지 사용
    img_path = list(uploaded.keys())[0]
    user_img = Image.open(img_path).convert('RGB')

    print(f"이미지 업로드 완료: {img_path}")
    print(f"크기: {user_img.size}")

    # 검출 수행
    results = model(user_img, conf=0.5)
    result_img = results[0].plot()

    # 결과 시각화
    fig, axes = plt.subplots(1, 2, figsize=(16, 8))
    axes[0].imshow(user_img)
    axes[0].set_title('Uploaded Image', fontsize=14, fontweight='bold')
    axes[0].axis('off')

    axes[1].imshow(cv2.cvtColor(result_img, cv2.COLOR_BGR2RGB))
    axes[1].set_title(f'The result: ({len(results[0].boxes)}object)',
                     fontsize=14, fontweight='bold')
    axes[1].axis('off')

    plt.tight_layout()
    plt.show()

    # 검출 상세 정보
    print("\n검출된 객체:")
    for i, box in enumerate(results[0].boxes):
        cls = int(box.cls[0].cpu().item())
        conf = box.conf[0].cpu().item()
        print(f"  {i+1}. {class_names[cls]}: {conf:.3f}")
else:
    print("업로드된 이미지가 없습니다.")